In [5]:
import numpy as np

In [6]:
0.002 / 10

0.0002

In [16]:

np.arange(start=0.0002, stop=0.002, step=0.0002)

array([0.0002, 0.0004, 0.0006, 0.0008, 0.001 , 0.0012, 0.0014, 0.0016,
       0.0018])

In [11]:
np.arange(1,10, 0.5)

array([1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. , 5.5, 6. , 6.5, 7. ,
       7.5, 8. , 8.5, 9. , 9.5])